In [1]:
import os
from bs4 import BeautifulSoup

In [2]:
slozka = "persony_cs"

In [3]:
def seznam_souboru(slozka):
    seznam = []
    for file in os.listdir(slozka):
        seznam.append(file)
    return(seznam)

In [4]:
seznam_person = seznam_souboru(slozka)
seznam_person[0:10]

['1000-ondrej-masek.html',
 '100019-kino-svetozor.html',
 '100038-ad-wood-sro.html',
 '100039-hypermarket-film-s-r-o.html',
 '100039-hypermarket-film-sro.html',
 '100039-hypermarket-film.html',
 '100076-punk-film-sro.html',
 '100076-punk-film.html',
 '10009-jan-kaspar.html',
 '1001-renata-pavlickova.html']

In [5]:
len(seznam_person)

35976

In [6]:
def jake_jsou_bio_udaje(slozka):
    bio = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='item-details')
            for item_detail in item_details:
                h3_tags = item_detail.find_all('h3')
                for h3_tag in h3_tags:
                    if h3_tag.get_text() not in bio:
                        bio.append(h3_tag.get_text())
    return(bio)

In [7]:
kolik = 1000

In [8]:
bios = jake_jsou_bio_udaje(slozka)
bios

['Alternativní jména',
 'Datum narození',
 'Místo narození',
 'Datum úmrtí',
 'Místo úmrtí']

In [9]:
import pandas as pd

In [10]:
def id_filmu(url):
    if "cs/film/" in url:
        id_filmu = url.split("/")[-2]
    else:
        id_filmu = 0
    return(id_filmu)

In [11]:
import re

In [12]:
def scrape_person(slozka, soubor):
    
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
            
    try:
    
        bio_udaje = {}

        jmeno = soup.find("h1").text.strip()

        id = soup.find('meta', {'property': 'og:url'})['content'].split("/")[-2]

        bio_udaje["Jméno"] = f"{jmeno} ({id})"
        
        bio_udaje["Id"] = id

        if soup.find(id="description-body-reducer-bio"):
                        
            bio_udaje["Životopis"] = soup.find(id="description-body-reducer-bio").text.strip()
            
        item_details = soup.find_all(class_='item-details')

        for item_detail in item_details:
                        
            if item_detail.find("div"):
                       
                divs = item_detail.find_all("div")

                for div in item_detail:
                    for b in bios:
                        if b in div.text:
                            bio_udaje[b] = div.text.replace(b,"").strip()
                            
            else:
                bio_udaje["Medailon"] = item_detail.text.strip()
        
        
        if soup.find(id="person-filmography-list-target"):
            bio_udaje["Osoba"] = "fyzická"
        if soup.find(id="company-filmography-list-target"):
            bio_udaje["Osoba"] = "právnická"
            
        if bio_udaje["Osoba"] == "fyzická":
            
            oceneni = soup.find_all(class_="credit-item")

            for o in oceneni:
                if (o.find("h3").text.strip().lower() == "vítěz"):
                    if ("čestný titul národní umělec" in o.find("p").text.lower()):
                        bio_udaje["Národní umělec"] = re.search("19\d{2}", o.text).group(0)
                    if ("čestný titul zasloužilý umělec" in o.find("p").text.lower()):
                        bio_udaje["Zasloužilý umělec"] = re.search("19\d{2}", o.text).group(0)
                
        bio_udaje = pd.DataFrame.from_dict([bio_udaje])

    except:
        bio_udaje = pd.DataFrame()
    
    return(bio_udaje)

In [13]:
test = scrape_person("persony_cs","22659-kuhnuv-detsky-pevecky-sbor.html")
test

,Jméno,Id,Alternativní jména,Osoba
0,Kühnův dětský sbor (22659),22659,Kühnův dětský pěvecký sbor,právnická


In [14]:
test = scrape_person("persony_cs","126270-frantisek-kubka.html")
test

,Jméno,Id,Medailon,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Alternativní jména,Osoba,Zasloužilý umělec
0,František Kubka (126270),126270,"dramatik, spisovatel, básník, novinář, prozaik...",04.03.1894,"Praha, Rakousko-Uhersko",07.01.1969,"Praha, Československo","A. K. Buk (pseudonym), Cumulus (pseudonym), ...",fyzická,1954


In [15]:
test = scrape_person("persony_cs","4842-rudolf-hrusinsky-st.html")
test

,Jméno,Id,Medailon,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Alternativní jména,Osoba,Národní umělec
0,Rudolf Hrušínský st. (4842),4842,"scenárista, herec, pedagog, režisér",17.10.1920,"Nová Včelnice u Kamenice nad Lipou, Českoslove...",13.04.1994,"Praha, Česká republika",Rudolf Böhm (rodné jméno),fyzická,1988


In [16]:
def scrape_persony(slozka):
    persony = pd.DataFrame()
    count = 0 
    for s in seznam_souboru(slozka)[:kolik]:
        count = count + 1
        print(f"done: {count} / total: {len(seznam_souboru(slozka)[:kolik])} / {s}")
        h = scrape_person(slozka, s)
        persony = pd.concat([persony, h])
    persony = persony.reset_index(drop=True)
    return(persony)

In [17]:
kolik = None

In [ ]:
persony = scrape_persony("persony_cs")

done: 1 / total: 35976 / 1000-ondrej-masek.html
done: 2 / total: 35976 / 100019-kino-svetozor.html
done: 3 / total: 35976 / 100038-ad-wood-sro.html
done: 4 / total: 35976 / 100039-hypermarket-film-s-r-o.html
done: 5 / total: 35976 / 100039-hypermarket-film-sro.html
done: 6 / total: 35976 / 100039-hypermarket-film.html
done: 7 / total: 35976 / 100076-punk-film-sro.html
done: 8 / total: 35976 / 100076-punk-film.html
done: 9 / total: 35976 / 10009-jan-kaspar.html
done: 10 / total: 35976 / 1001-renata-pavlickova.html
done: 11 / total: 35976 / 10010-ales-zabojnik.html
done: 12 / total: 35976 / 100116-tomas-dalecky.html
done: 13 / total: 35976 / 100137-film-factory-sro.html
done: 14 / total: 35976 / 100138-film-factory-cz-sro.html
done: 15 / total: 35976 / 100139-vojtech-matyas-jirovec.html
done: 16 / total: 35976 / 100156-josef-gabriel.html
done: 17 / total: 35976 / 100196-pegasos-film.html
done: 18 / total: 35976 / 100197-sisaart.html
done: 19 / total: 35976 / 1002-jakub-zich.html
done: 20

done: 147 / total: 35976 / 100826-snake-catcher-sro.html
done: 148 / total: 35976 / 100826-snake-catcher.html
done: 149 / total: 35976 / 10083-iveta-matyasova.html
done: 150 / total: 35976 / 100838-filmovy-ustav.html
done: 151 / total: 35976 / 100839-bionaut-animation.html
done: 152 / total: 35976 / 100839-bionaut-docs.html
done: 153 / total: 35976 / 100839-bionaut-films.html
done: 154 / total: 35976 / 100839-bionaut-s-r-o.html
done: 155 / total: 35976 / 100839-bionaut-sro.html
done: 156 / total: 35976 / 100839-bionaut.html
done: 157 / total: 35976 / 10084-vi-de-kvartet.html
done: 158 / total: 35976 / 10085-marian-lejava.html
done: 159 / total: 35976 / 10086-peter-lipa.html
done: 160 / total: 35976 / 10087-andrea-nemethova.html
done: 161 / total: 35976 / 100876-mat-film-resort.html
done: 162 / total: 35976 / 100876-mat-film-sro.html
done: 163 / total: 35976 / 100876-mat-film.html
done: 164 / total: 35976 / 10089-alexandra-dubovska.html
done: 165 / total: 35976 / 100896-marie-vazquezova

done: 296 / total: 35976 / 102166-richard-dvorak.html
done: 297 / total: 35976 / 102179-innogy-ceska-republika-as.html
done: 298 / total: 35976 / 102179-innogy-ceska-republika.html
done: 299 / total: 35976 / 102179-innogy.html
done: 300 / total: 35976 / 102179-rwe-ceska-republika-as.html
done: 301 / total: 35976 / 102179-rwe-ceska-republika.html
done: 302 / total: 35976 / 102179-rwe-energie-ceskeho-filmu.html
done: 303 / total: 35976 / 102179-rwe-transgas-a-s.html
done: 304 / total: 35976 / 102179-rwe-transgas.html
done: 305 / total: 35976 / 102179-rwe.html
done: 306 / total: 35976 / 102180-fc-comp-cz-s-r-o.html
done: 307 / total: 35976 / 102182-rudolf-ulrich.html
done: 308 / total: 35976 / 102183-atelier-wengeroffilm-berlin.html
done: 309 / total: 35976 / 102183-wengeroffilm-berlin.html
done: 310 / total: 35976 / 102184-cinemania-sro.html
done: 311 / total: 35976 / 102184-cinemania.html
done: 312 / total: 35976 / 102186-infinity-prague-ltd-as.html
done: 313 / total: 35976 / 102186-inf

done: 438 / total: 35976 / 102796-magic-box-as.html
done: 439 / total: 35976 / 1028-soubor-koksetau.html
done: 440 / total: 35976 / 102836-pet-halmen.html
done: 441 / total: 35976 / 102896-gerard-vandenberg.html
done: 442 / total: 35976 / 1029-jiri-vitinger.html
done: 443 / total: 35976 / 102918-ateliery-gaumont-franco-film-aubert-nice.html
done: 444 / total: 35976 / 102918-gaumont-franco-film-aubert-nice.html
done: 445 / total: 35976 / 102976-2landa-s-r-o.html
done: 446 / total: 35976 / 102982-actors-runway-agency-sro.html
done: 447 / total: 35976 / 102982-actors-runway-agency.html
done: 448 / total: 35976 / 102997-pandora-cinema.html
done: 449 / total: 35976 / 1030-svatava-sophova.html
done: 450 / total: 35976 / 103017-artcorp-sro.html
done: 451 / total: 35976 / 103017-artcorp.html
done: 452 / total: 35976 / 103036-ateliery-gaumont-franco-film-aubert-pariz.html
done: 453 / total: 35976 / 103036-gaumont-franco-film-aubert-pariz.html
done: 454 / total: 35976 / 103039-blue-sky-film-dist

done: 582 / total: 35976 / 103819-s-films-sro.html
done: 583 / total: 35976 / 103819-starlite-films-s-r-o.html
done: 584 / total: 35976 / 103836-first-film-s-r-o.html
done: 585 / total: 35976 / 103840-independent-heads-s-r-o.html
done: 586 / total: 35976 / 103841-filmove-ateliery-s-r-o.html
done: 587 / total: 35976 / 103841-filmove-ateliery-sro.html
done: 588 / total: 35976 / 103841-filmove-ateliery-zlin.html
done: 589 / total: 35976 / 10386-milos-zverina.html
done: 590 / total: 35976 / 103876-songbird-pictures.html
done: 591 / total: 35976 / 103877-legende-film.html
done: 592 / total: 35976 / 103879-ema-svandova-kadlecova.html
done: 593 / total: 35976 / 103880-atelier-ceskoslovenskeho-filmoveho-ustavu-brno.html
done: 594 / total: 35976 / 103896-jihomoravsky-kraj.html
done: 595 / total: 35976 / 1039-jiri-cerny.html
done: 596 / total: 35976 / 103916-taiga-films-b-v-amsterdam.html
done: 597 / total: 35976 / 103920-studio-podium.html
done: 598 / total: 35976 / 103922-josef-ocman.html
done

...

done: 35139 / total: 35976 / 94376-ipon-jiri-kuba.html
done: 35140 / total: 35976 / 94376-ipon-stunts.html
done: 35141 / total: 35976 / 94376-ipon.html
done: 35142 / total: 35976 / 9438-rafael-eci-skurka.html
done: 35143 / total: 35976 / 9438-rafael-skurka.html
done: 35144 / total: 35976 / 9439-frantisek-rakicky-suli.html
done: 35145 / total: 35976 / 9439-frantisek-rakicky.html
done: 35146 / total: 35976 / 9439-frantisek-rakitski.html
done: 35147 / total: 35976 / 9439-frantisek-suli-rakicky.html
done: 35148 / total: 35976 / 9439-suli-rakicky.html
done: 35149 / total: 35976 / 94396-fakulta-multimedialnich-komunikaci.html
done: 35150 / total: 35976 / 94396-fmk-utb.html
done: 35151 / total: 35976 / 944-jan-brejla.html
done: 35152 / total: 35976 / 9440-robert-kovacs.html
done: 35153 / total: 35976 / 9442-oga-kovacovicova.html
done: 35154 / total: 35976 / 9443-jarmila-schniererova.html
done: 35155 / total: 35976 / 9444-erika-gadusova.html
done: 35156 / total: 35976 / 9445-ladislav-szabo.htm

done: 35285 / total: 35976 / 9505-vlasta-styblikova.html
done: 35286 / total: 35976 / 95056-max-urban.html
done: 35287 / total: 35976 / 95057-atelier-wetebu.html
done: 35288 / total: 35976 / 95057-weteb.html
done: 35289 / total: 35976 / 95058-frantisek-filip.html
done: 35290 / total: 35976 / 95059-vaclav-kolar-modrany.html
done: 35291 / total: 35976 / 9506-jindrich-vodicka.html
done: 35292 / total: 35976 / 9507-otakar-vavra.html
done: 35293 / total: 35976 / 9508-lada-vackova.html
done: 35294 / total: 35976 / 951-jitka-hejbalova.html
done: 35295 / total: 35976 / 9510-josef-vanis.html
done: 35296 / total: 35976 / 9511-ales-voleman.html
done: 35297 / total: 35976 / 9512-milan-pavlik.html
done: 35298 / total: 35976 / 9513-premysl-prokop.html
done: 35299 / total: 35976 / 95136-e-motion-film-ljubljana.html
done: 35300 / total: 35976 / 9515-josef-pesek.html
done: 35301 / total: 35976 / 9516-jiri-ployhar.html
done: 35302 / total: 35976 / 9517-j-a-novotny.html
done: 35303 / total: 35976 / 9518-

done: 35431 / total: 35976 / 96540-centrum-tvorby-pro-deti-a-mladez.html
done: 35432 / total: 35976 / 96541-centrum-publicistiky-a-dokumentu.html
done: 35433 / total: 35976 / 96548-centrum-hudebni-a-divadelni-tvorby.html
done: 35434 / total: 35976 / 96550-centrum-publicistiky-dokumentaristiky-a-vzdelavani.html
done: 35435 / total: 35976 / 96553-centrum-dramaticke-tvorby.html
done: 35436 / total: 35976 / 9657-wilhelm-grimm.html
done: 35437 / total: 35976 / 9658-jacob-grimm.html
done: 35438 / total: 35976 / 96596-centrum-publicistiky-a-dokumentaristiky-a-vzdelavani-praha.html
done: 35439 / total: 35976 / 96596-centrum-publicistiky-a-dokumentu.html
done: 35440 / total: 35976 / 96621-patmat-sro.html
done: 35441 / total: 35976 / 96636-ceskoslovenske-armadni-filmove-studio.html
done: 35442 / total: 35976 / 96636-ceskoslovensky-armadni-film.html
done: 35443 / total: 35976 / 96636-studio-caf.html
done: 35444 / total: 35976 / 96638-eva-matouskova.html
done: 35445 / total: 35976 / 96664-filmove-

done: 35572 / total: 35976 / 97236-media-nest.html
done: 35573 / total: 35976 / 97257-statutarni-mesto-ostrava.html
done: 35574 / total: 35976 / 97259-charlotte-ella-gottova.html
done: 35575 / total: 35976 / 9726-vera-kuchtova.html
done: 35576 / total: 35976 / 97260-cesko-nemecky-fond-budoucnosti.html
done: 35577 / total: 35976 / 9728-lubomir-lipsky.html
done: 35578 / total: 35976 / 9729-josef-laufer-dab.html
done: 35579 / total: 35976 / 9729-josef-laufer.html
done: 35580 / total: 35976 / 973-zdena-kocova.html
done: 35581 / total: 35976 / 9730-karel-mario-martinek.html
done: 35582 / total: 35976 / 9730-karel-martinek.html
done: 35583 / total: 35976 / 9731-vilem-persy.html
done: 35584 / total: 35976 / 97316-tony-kumpera.html
done: 35585 / total: 35976 / 9732-hana-cizkova.html
done: 35586 / total: 35976 / 9732-hanka-cizkova.html
done: 35587 / total: 35976 / 9733-petr-peknic.html
done: 35588 / total: 35976 / 9734-nikola-navratil.html
done: 35589 / total: 35976 / 97341-loutkovy-film-praha.

done: 35720 / total: 35976 / 98397-bkp-film-sro.html
done: 35721 / total: 35976 / 98397-kviff-production-s-r-o.html
done: 35722 / total: 35976 / 984-pavel-liska.html
done: 35723 / total: 35976 / 9840-alica-dvorska.html
done: 35724 / total: 35976 / 9841-martin-jankovic.html
done: 35725 / total: 35976 / 98416-major-k-international-s-r-o.html
done: 35726 / total: 35976 / 9842-michal-gubala.html
done: 35727 / total: 35976 / 9843-igor-gubala.html
done: 35728 / total: 35976 / 98437-bio-illusion-sro.html
done: 35729 / total: 35976 / 98437-bio-illusion.html
done: 35730 / total: 35976 / 98438-ceska-produkcni-invest-a-s.html
done: 35731 / total: 35976 / 9844-stefan-nagy.html
done: 35732 / total: 35976 / 9845-miroslav-rapcan.html
done: 35733 / total: 35976 / 9846-viera-rajtarova.html
done: 35734 / total: 35976 / 9847-maria-chmelova.html
done: 35735 / total: 35976 / 98477-nora-houfova.html
done: 35736 / total: 35976 / 9848-rudolf-grznar.html
done: 35737 / total: 35976 / 98482-axman-production-spol

In [ ]:
persony.sample(15)

In [ ]:
persony.to_json(os.path.join("data", "persony_raw.json"), orient="records")